### 분할 이미지 데이터 모으기

In [1]:
# !pip install tqdm
from shutil import copyfile
from glob import glob
from tqdm import tqdm
import os

In [2]:
def movefiles(filepath, movepath):
    copyfile(filepath, movepath)

In [3]:
train_paths = glob(os.path.join('data', 'train', '*', '*.jpg'))
test_paths = glob(os.path.join('data', 'test', '*', '*.jpg'))

In [4]:
len(train_paths), len(test_paths)

(95454, 8000)

In [5]:
train_rename = ['data\\train_all\\' + train_path.split('\\')[-1] for train_path in train_paths]
test_rename = ['data\\test_all\\' + test_path.split('\\')[-1] for test_path in test_paths]

In [6]:
len(train_rename), len(test_rename)

(95454, 8000)

In [7]:
print(train_paths[0], train_rename[0])

data\train\0\0017qu55dk.jpg data\train_all\0017qu55dk.jpg


In [8]:
for i in tqdm(range(len(train_paths))):
    copyfile(train_paths[i], train_rename[i])


100%|██████████| 95454/95454 [24:18<00:00, 65.43it/s]


In [9]:
for i in tqdm(range(len(test_paths))):
    copyfile(test_paths[i], test_rename[i])

100%|██████████| 8000/8000 [01:04<00:00, 124.80it/s]


## Using Colab
##### - 통합 이미지 데이터 구글 업로드 및 압축 풀기

In [ ]:
import pandas as pd
import numpy as np

In [1]:
# !unzip /content/drive/"My Drive"/Dacon/train_all.zip -d /content/drive/"My Drive"/Dacon/train/
# !unzip /content/drive/"My Drive"/Dacon/test_all.zip -d /content/drive/"My Drive"/Dacon/test/

In [ ]:
train_csv = '/content/drive/My Drive/Dacon/train.csv'
sample_sub_csv = '/content/drive/My Drive/Dacon/sample_submission.csv'

In [ ]:
train = pd.read_csv(train_csv)
sample = pd.read_csv(sample_sub_csv)

In [ ]:
# CategoryId No.11 = Leggings 존재 X
# train['CategoryId'].value_counts()